# Ionic (Phonon) Dielectric Constants in Free-Standing 2D and Bulk Cubic Perovskites

As the fast-moving electrons response to the vibrating ions, it can also induce an imbalance of charge distributions in the solid, which provides additional contributions to dielectric screening of charge carriers. According to the classical prescription, the ionic contributions to the static dielectric permittivity tensor can be obtained by summing the contributions from individual phonon mode in a solid as $\varepsilon_{\alpha\beta}^{\mbox{ion}}=\sum_{\omega_\lambda^2>0}\varepsilon^{\mbox{ion}}_{\alpha\beta}(\omega_\lambda)$, in which the mode-dependent dielectric permittivity tensor
\begin{equation}\label{eq:mode_dielectric}
   \varepsilon^{\mbox{ion}}_{\alpha\beta}(\omega_\lambda)=\frac{\bar{Z}_{\lambda\alpha}^{*}\bar{Z}_{\lambda\beta}^{*}}{V\varepsilon_0 m_0\omega_\lambda^2},
\end{equation}
where $V$ is the volume per unit cell, $m_0=1$ a.m.u., $\varepsilon_0$ is the  permittivity of the free-space, and $\bar{Z}_{\lambda\alpha}^{*}$ being the  Born effective charge along the $\alpha$-Cartesian direction for the $\lambda$-th phonon mode, which can be calculated from the atomic Born charges as
\begin{equation}\label{eq:mode_born_charge}
    \bar{Z}_{\lambda\alpha}^{*}=\sum_{i\gamma}\left(\sqrt{m_0/m_i}v_{\lambda i\gamma}\right)Z_{i\alpha\gamma}^{*},
\end{equation}
in which $Z^{*}_i$ is the Born effective charge tensor for atom $i$, $m_i$ being its atomic mass and $v_{\lambda i\gamma}$ is the normalized component of phonon eigenvector for mode $\lambda$ on atom $i$ along the $\gamma$ Cartesian direction. This shows that  large ionic dielectric permittivity in materials can be contributed by: (a) large mode effective charge $\bar{Z}_\lambda$, which, according to the expression for $\varepsilon^{\mbox{ion}}_{\alpha\beta}(\omega_\lambda)$, predominantly arises from optical modes that involve ions with opposite charges vibrating in the opposite directions, (b) the optical phonon having low frequency $\omega_{\lambda}$ and (c) low crystal symmetry that increases the total number of vibrational modes. Fundamentally, these factors are interdependent and determined by the geometries and strengths of the chemical bondings, as well as the electronic structures of the materials.  

As such, it is of our fundamental interest to examine how the formation of free-standing 2D perovskite membranes will change $\varepsilon_{\mbox{ion}}$ from their bulk counterparts, that is resulted from the interplay between changes in the materials both structurally and electronically. Furthermore, it should be noted that the dielectric permittivity  for 2D perovskites that is  directly calculated from  DFT include the contributions from the non-vanishing electronic interactions between the periodic images of the 2D materials that are separated by the vacuum slabs of finite length in the simulation supercell. Practically, this  contribution from the finite vacuum slab may be asymptotically corrected to the  limit of infinite vacuum, in order to obtain  more reasonable values for the dielectric permittivities of the 2D materials, and these corrections can be achieved via the following relationships:
\begin{equation}\label{eq:epsilon_inplane_outplane}
\varepsilon_{2D}^\parallel=\frac{c}{t}\left(\varepsilon_{S}^\parallel-1\right)+1,
\qquad \varepsilon_{2D}^\perp = \left[ \frac{c}{t} \left(\frac{1}{\varepsilon_S^\perp}-1\right)+1\right]^{-1}.
\end{equation}
Here, $\varepsilon_{S}$ and $\varepsilon_{2D}$ are the dielectric permittivities of 2D solid directly obtained from DFT, and that with the effect of finite vacuum asymptotically corrected, respectively. In particular, the in-plane  $\varepsilon_{S}^\parallel=(\varepsilon_{xx}+\varepsilon_{yy})/2$ and out-of-plane $\varepsilon_{S}^\perp=\varepsilon_{zz}$ components of the dielectric tensors are corrected separately, whereby $t$ and $c$ in the above two equations are the thickness of the optimized 2D structure and the supercell length along the  normal direction to the 2D structure, respectively. 


Note: We found that the predominant contribution to the ionic dielectric constants come from the in-plane components $\varepsilon_{2D}^\parallel$ where as the out-of-plane components $\varepsilon_{2D}^\perp$ are completely suppressed. Hence here we will just show the data for $\varepsilon_{2D}^\parallel$.

In [1]:
try:
    from ase.db import connect
except ImportError:
    !pip install ase
    from ase.db import connect

try:
    import plotly 
except ImportError:
    !pip install plotly
import plotly.graph_objects as go

from utils import *
from element import *
import pandas as pd
import numpy as np
import math

In [2]:
def non_zero(tensor):
    return (tensor[0][0] != 0.0) and (tensor[1][1] != 0.0) and (tensor[2][2] != 0.0)

def trace(tensor):
    return (tensor[0][0] + tensor[1][1] + tensor[2][2]) / 3.0

def trace_inplane(tensor):
    return (tensor[0][0] + tensor[1][1]) / 2.0

def corrected_trace_inplane(db,sys_name):
    row = db.get(selection=[('uid', '=', sys_name)])
    structure_2d = row.toatoms()
    supercell_c = structure_2d.get_cell_lengths_and_angles()[2]
    all_z_positions = [x[-1]/supercell_c for x in structure_2d.get_positions()]
    all_z_positions = all_z_positions - np.round(all_z_positions)
    all_z_positions = [z * supercell_c for z in all_z_positions]    
    slab_thick = max(all_z_positions) - min(all_z_positions)
    twod_dielectric_tensor = row.data['dielectric_ionic_tensor']
    twod_epsilon_inplane = trace_inplane(twod_dielectric_tensor)
    twod_epsilon_inplane = (supercell_c / slab_thick) * (twod_epsilon_inplane - 1.0) + 1.0
    return twod_epsilon_inplane

In [7]:
bulk_db=connect('2dpv_set_bulk.db')
all_data_dict={}
database_counter=0
for orient in ['110', '100', '111']:
    for term in termination_types[orient]:
        this_db = connect('2dpv_set_'+orient+'_'+term+'.db')

        for i in range(len(A_site_list)): 
            for count_a, a in enumerate(A_site_list[i]):
                for b in B_site_list[i]:
                    for c in C_site_list[i]:
                
                        system_name = a + b + c
                        uid = system_name + '3_pm3m'
                        row = bulk_db.get(selection=[('uid', '=', uid)])
                        try:
                            bulk_dielectric_tensor = row.data['dielectric_ionic_tensor']
                        except:
                            continue

                        bulk_epsilon = None
                        if non_zero(bulk_dielectric_tensor):
                            bulk_epsilon = trace(bulk_dielectric_tensor)    
                            
                        for thick in [3,5,7,9]:  
                            uid_2d = system_name + '3_' + str(orient) + "_" + str(term) + "_" + str(thick)
                            twod_epsilon_inplane = None
                            try:
                                twod_epsilon_inplane = corrected_trace_inplane(this_db,uid_2d)
                            except:
                                pass
                            
                            if (bulk_epsilon is not None and twod_epsilon_inplane is not None):
                                this_data_dict = {
                                    'chemical_class':i+1,
                                    'system':uid.split("_")[0],
                                    'orientation':orient,
                                    'termination':term,
                                    'thickness':thick,
                                    'bulk_epsilon':bulk_epsilon,
                                    'twod_epsilon_inplane':twod_epsilon_inplane,
                                }
                                all_data_dict[database_counter]=this_data_dict
                                database_counter+=1
        this_db=None      
        
df = pd.DataFrame(all_data_dict).T

In [19]:
chemical_classes = df['chemical_class'].unique().tolist()
crystallographic_orientations = df['orientation'].unique().tolist()
terminations = df['termination'].unique().tolist()
thicknesses = df['thickness'].unique().tolist()

df_categorized = {}
df_categorized_thick_dependent={}
for cc in chemical_classes:
    for cd in crystallographic_orientations:
        for term in terminations:
            
            this_key = str(cc)+'_'+str(cd)+'_'+str(term)
            this_set = df[(df['chemical_class']==cc) & (df['orientation']==cd) & (df['termination']==term)]
            if not this_set.empty:
                df_categorized_thick_dependent[this_key] = this_set
            this_set=None
            
            for thickness in [3,5,7,9]:
                key = str(cc)+'_'+str(cd)+'_'+str(term)+'_'+str(thickness)
                this_set = df[(df['chemical_class']==cc) & (df['orientation']==cd) & (df['termination']==term) & (df['thickness']==thickness)]
            
                if not this_set.empty:
                    df_categorized[key] = this_set
                this_set=None

In [20]:
df_reshaped={}
common_col_dicts={}
for df_item in df_categorized_thick_dependent:
    common_rows=[]
    common_cols=[]
    common_rows=sorted(list(set().union(common_rows,list(df_categorized_thick_dependent[df_item].thickness))))
    common_cols=sorted(list(set().union(common_cols,list(df_categorized_thick_dependent[df_item].system))))
    
    df_common = pd.DataFrame(np.nan, index=common_rows, columns=common_cols)
    
    #this is a dumm way to get the number from the original table and repopulated into the reshaped one,
    # but it's probably easier to understand what the hell is going on.
    cc = df_item.split("_")[0]
    cd = df_item.split("_")[1]
    t = df_item.split("_")[2]

    for col in common_cols:
        for row in common_rows:
            value = None
            try:
                value = df.loc[(df['chemical_class'] == int(cc)) & 
                               (df['orientation'] == cd)    &
                               (df['termination'] == t)     &
                               (df['system'] == col)        &
                               (df['thickness'] == row), 'twod_epsilon_inplane'].iat[0]
            except:
                pass
            if value is not None:
                df_common[col][row] = value
    df_reshaped[df_item]=df_common
    common_col_dicts[df_item]=common_cols

## Comparing Ionic Dielectric Constants for Bulk and 2D Perovskites

In [15]:
import copy
sorted_keys = list(sorted(df_categorized.keys()))
fig = go.Figure()

button_list=[]
display_this=[False for _ in sorted_keys]

for i,key in enumerate(sorted_keys):
    fig.add_trace(go.Scatter(x=df_categorized[key]['bulk_epsilon'],
                             y=df_categorized[key]['twod_epsilon_inplane'],
                             #visible=True,
                             opacity=0.9,
                             marker=dict(size=12,color='#6FB98F'),
                             marker_symbol='circle',#name=col,
                             text=df_categorized[key]['system'],
                             name=key
                            )
                 )
    _display_this = copy.deepcopy(display_this)
    _display_this[i] = True
    button_list.append(dict(label = key,
                            method = 'update',
                              args = [{'visible': _display_this},
                                      {'showlegend':False}]))



fig.update_layout(width=900, height=800,
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=button_list)])
    
fig.update_xaxes(title_text='$\\varepsilon_{bulk}$',type='log')
fig.update_yaxes(title_text='$\\varepsilon_{2D,\\parallel}$',type='log')
#fig.update_layout(yaxis_range=[-0.6,0.3],xaxis_range=[0.5,1.2])
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)

fig.show()

## Size-Dependent In-Plane Ionic Dielectric Constants for 2D Perovskites

In [22]:
from collections import defaultdict
    
try:
    import bokeh
except ImportError:
    !pip install bokeh
    
from bokeh.plotting import show, figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook 
output_notebook()

#This version has no dropdown menu to select the system, so need to be typed in here
chemical_class = '1_100_BO2'

data = defaultdict(list)
for col in df_reshaped[chemical_class]:
    data['system'].append(col)
    #print(common_rows)
    data['thicknesses'].append(common_rows)
    data['inplane_epsilon'].append(df_reshaped[chemical_class][col].values)
    
source = ColumnDataSource(data=dict(system=data['system'],
                                    thicknesses=data['thicknesses'],
                                    inplane_epsilon=data['inplane_epsilon']))
p = figure(plot_height=400, y_axis_type="log")
p.multi_line(xs='thicknesses', ys='inplane_epsilon', 
             line_width=2, line_color='#dd0000', line_alpha=0.1,
             hover_line_color='#dd0000', hover_line_alpha=1.0,
             source=source)
p.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
    ('system', '@system')
]))
show(p)

Loading BokehJS ...